# Setup

### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install Dependencies

In [ ]:
# Needed for converting tif to png
!sudo apt-get update

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,512 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:1

### Imports & Global Variables

In [ ]:
import os
import glob


remote_code_path = "/content/drive/MyDrive/white_light"
train_images = "/content/drive/MyDrive/unet_training/val/images"
train_gts = "/content/drive/MyDrive/unet_training/val/masks"
test_images = "/content/drive/MyDrive/unet_training/val/images"
test_gts = "/content/drive/MyDrive/unet_training/val/masks"
local_code_path = "/content/"+os.path.basename(remote_code_path)
first_image_index = 1
last_image_index = 8

### Copy code into working directory

In [ ]:
os.system(f"cp -r {remote_code_path} ./")

0

# Prepare Data

### Fetch and Parse VesselNet Data

In [ ]:
# Clear the code's data directory
os.system(f"rm {local_code_path}/DRIVE/training/images/*")
os.system(f"rm {local_code_path}/DRIVE/training/1st_manual/*")

imgs = glob.glob(f"{train_images}/*")
print(len(imgs))
gts = glob.glob(f"{train_gts}/*")

imgs.sort()
gts.sort()

for i in range(first_image_index, last_image_index+1):
  # Create n masks (n is the number of images to train on)
  os.system(f"cp {local_code_path}/DRIVE/training/mask/20_training_mask.png {local_code_path}/DRIVE/training/mask/{i:02d}_training_mask.png")
  
  # Resize both images and groundtruth to UNet's size (512x512)
  print(imgs[i-first_image_index])
  os.system(f"ffmpeg -y -i {imgs[i-first_image_index]} -s 512x512 {local_code_path}/DRIVE/training/images/{i:02d}_training.jpg")
  os.system(f"ffmpeg -y -i {gts[i-first_image_index]} -s 512x512 {local_code_path}/DRIVE/training/1st_manual/{i:02d}_manual1.png")




8
/content/drive/MyDrive/unet_training/val/images/04_training.jpg
/content/drive/MyDrive/unet_training/val/images/11.png
/content/drive/MyDrive/unet_training/val/images/19.png
/content/drive/MyDrive/unet_training/val/images/24.png
/content/drive/MyDrive/unet_training/val/images/3.png
/content/drive/MyDrive/unet_training/val/images/30.png
/content/drive/MyDrive/unet_training/val/images/4.png
/content/drive/MyDrive/unet_training/val/images/8.png


In [ ]:
# Clear the code's data directory
os.system(f"rm {local_code_path}/DRIVE/test/images/*")
os.system(f"rm {local_code_path}/DRIVE/test/1st_manual/*")
os.system(f"rm {local_code_path}/DRIVE/test/1st_manual/*")

imgs = glob.glob(f"{test_images}/*")
print(len(imgs))
gts = glob.glob(f"{test_gts}/*")

imgs.sort()
gts.sort()

for i in range(first_image_index, last_image_index+1):
  # Create n masks (n is the number of images to train on)
  os.system(f"cp {local_code_path}/DRIVE/test/mask/21_test_mask.png {local_code_path}/DRIVE/test/mask/{i+20:02d}_test_mask.png")
  
  # Resize both images and groundtruth to UNet's size (512x512)
  os.system(f"ffmpeg -y -i {imgs[i-first_image_index]} -s 512x512 {local_code_path}/DRIVE/test/images/{i+20:02d}_test.jpg")
  os.system(f"ffmpeg -y -i {gts[i-first_image_index]} -s 512x512 {local_code_path}/DRIVE/test/1st_manual/{i+20:02d}_manual1.png")




8


### Prepare Dataset

In [ ]:
# This extracts all testing and training patches and saves them into an h5 file to be used by the network
!cd white_light && python3 prepare_datasets_DRIVE.py

original image: 
ground truth name: 17_manual1.png
border masks name: 17_training_mask.png
original image: 
ground truth name: 04_manual1.png
border masks name: 04_training_mask.png
original image: 
ground truth name: 19_manual1.png
border masks name: 19_training_mask.png
original image: 
ground truth name: 02_manual1.png
border masks name: 02_training_mask.png
original image: 
ground truth name: 06_manual1.png
border masks name: 06_training_mask.png
original image: 
ground truth name: 10_manual1.png
border masks name: 10_training_mask.png
original image: 
ground truth name: 09_manual1.png
border masks name: 09_training_mask.png
original image: 
ground truth name: 20_manual1.png
border masks name: 20_training_mask.png
original image: 
ground truth name: 08_manual1.png
border masks name: 08_training_mask.png
original image: 
ground truth name: 03_manual1.png
border masks name: 03_training_mask.png
original image: 
ground truth name: 01_manual1.png
border masks name: 01_training_mask.png

# Run Training

In [ ]:
!cd white_light&& python3 src/retinaNN_training.py


train images/masks shape:
(28, 1, 503, 512)
train images range (min-max): 0.0 - 1.0
train masks are within 0-1

tcmalloc: large alloc 4128768000 bytes == 0x20d52000 @  0x7f1d5a3ca1e7 0x7f1d57cdb0ce 0x7f1d57d31cf5 0x7f1d57d31f4f 0x7f1d57dd4673 0x5936cc 0x548c51 0x51566f 0x549576 0x593fce 0x5118f8 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f1d59fc7c87 0x5b621a
tcmalloc: large alloc 4128768000 bytes == 0x116ed2000 @  0x7f1d5a3ca1e7 0x7f1d57cdb0ce 0x7f1d57d31cf5 0x7f1d57d31f4f 0x7f1d57dd4673 0x5936cc 0x548c51 0x51566f 0x549576 0x593fce 0x5118f8 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f1d59fc7c87 0x5b621a
patches per full image: 8000

train PATCHES images/masks shape:
(224000, 1, 48, 48)
train PATCHES images range (min-max): 0.0 - 1.0
2022-06-06 01:04:08.188017: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_F

In [ ]:
!cd white_light&& python3 src/retinaNN_predict.py

./lib/pre_processing.py:65: RuntimeWarning: invalid value encountered in true_divide
  imgs_normalized[i] = ((imgs_normalized[i] - np.min(imgs_normalized[i])) / (np.max(imgs_normalized[i])-np.min(imgs_normalized[i])))*255

the side H is not compatible with the selected stride of 5
img_h 512, patch_h 48, stride_h 5
(img_h - patch_h) MOD stride_h: 4
So the H dim will be padded with additional 1 pixels
the side W is not compatible with the selected stride of 5
img_w 512, patch_w 48, stride_w 5
(img_w - patch_w) MOD stride_w: 4
So the W dim will be padded with additional 1 pixels
new full images shape: 
(20, 1, 513, 513)

test images shape:
(20, 1, 513, 513)

test mask shape:
(20, 1, 512, 512)
test images range (min-max): 0.0 - 1.0
test masks are within 0-1

Number of patches on h : 94
Number of patches on w : 94
number of patches per image: 8836, totally for this dataset: 176720
tcmalloc: large alloc 3257303040 bytes == 0x2e404000 @  0x7f1a0030c1e7 0x7f19fdc5d0ce 0x7f19fdcb3cf5 0x7f19fdcb